#### 导入相关包

In [23]:
from used_packages import *

## 一、 读取数据
### 1、设置数据路径

In [24]:
data_path = r'H:\Solomon\EEG_WORK\data\feature\ctm_after_rhythm_without_processing'
result_path = r'H:\Solomon\EEG_WORK\result\0322_NB_ctm_result_without_process_rhythm'
os.makedirs(result_path)

### 2 读取ctm特征数据

In [25]:
def read_ctm(r=1):
    imf_list = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    # rhythm_list = ['delta', 'theta', 'alpha', 'beta', 'gamma']

    df = pd.DataFrame()

    for imf in imf_list:
        feature_path = f"{data_path}\\r={r}\\{imf}"
        tmp_df = pd.DataFrame()
        for file in os.listdir(feature_path):
            tmp = pd.read_csv(f"{feature_path}\\{file}", index_col=0)
            pid = file[11:13]
            tmp['pid'] = pid
            tmp_df = tmp_df.append(tmp)
        tmp_df['imf'] = imf
        df = df.append(tmp_df)
    return df
df = read_ctm()
df

,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,...,PPO2h,PPO6h,PPO10h,P10,I1,OI1h,OI2h,I2,pid,imf
C12,715,941,462,688,840,692,631,392,1017,1479,...,683,414,585,710,626,588,592,689,01,delta
C11,556,735,180,601,693,797,650,294,959,706,...,381,465,337,431,451,514,533,553,01,delta
C12,1223,1340,775,1181,722,860,805,528,823,796,...,722,561,499,646,647,584,542,577,01,delta
C12,378,619,410,323,483,559,491,271,925,1543,...,399,248,275,270,207,206,212,223,01,delta
C11,657,832,511,700,614,604,529,562,1168,1200,...,590,813,622,530,398,431,475,505,01,delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C21,1634,1868,1150,1724,2315,1603,1394,1182,2521,2645,...,1048,693,844,1018,637,633,726,814,20,gamma
C21,1650,1834,1094,1652,2359,1605,1412,1214,2476,2815,...,987,594,802,962,633,643,789,852,20,gamma
C21,1676,1718,1052,1534,2312,1439,1549,1255,2422,2713,...,1106,579,767,992,630,639,722,774,20,gamma
C21,1706,1762,1213,1670,2474,1465,1499,1215,2484,2665,...,1145,534,676,898,727,679,708,732,20,gamma


## 分类实验
#### 一些分类参数

In [26]:
# 半径情况
r_list = [1,2,10]

# 节律情况
imf_list = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# 通道选择情况
channel_need = ['FFC1h', 'AFF2h', 'FCC4h', 'TTP8h', 'FCC1h', 'F1',
                'FT10', 'C5', 'FFT7h', 'F3', 'AFp2']

# 标签转换，C2 为大类，C4 为小类
# cls_label_trans = {'C2':{'C11': 1, 'C12': 1, 'C21': 2, 'C22': 2},
#                    'C4':{'C11': 1, 'C12': 2, 'C21': 3, 'C22': 4}}

cls_label_trans = {'C2':{'C11': 1, 'C12': 1, 'C21': -1, 'C22': -1},
                   'C41':{'C11': 1, 'C12': -1},
                   'C42':{'C21': 1, 'C22': -1}}

# 内外的交叉验证
inner_cv_C2 = KFold(n_splits=5, shuffle=True, random_state=99)
outer_cv_C2 = KFold(n_splits=3, shuffle=True, random_state=1)

inner_cv_C4 = KFold(n_splits=10, shuffle=True, random_state=98)
outer_cv_C4 = KFold(n_splits=10, shuffle=True, random_state=2)



# 分类器
clf = GaussianNB()

#### ④ 单人5通道单节律分类
##### ④.① Prefronta Cortex

In [27]:
param_grid = [
                {'priors': [None]}
              ]

Prefrontal_Cortex = ['Fpz','Fp1','Fp2','AFp1', 'AFp2']
# 按照半径展开
for r in r_list:
    df = read_ctm(r)

    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        ans_df = pd.DataFrame(columns=imf_list, index=range(1,21))

        if cls_key == 'C2':
            inner_cv = inner_cv_C2
            outer_cv = outer_cv_C2
        else:
            inner_cv = inner_cv_C4
            outer_cv = outer_cv_C4
        # 按照节律展开
        for imf in imf_list:
            # 按照pid 展开
            for pid in range(1, 21):
                sys.stdout.write(f"\r r {r} class {cls_key} imf {imf} pid {pid}")
                X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]

                X_Y_data = X_Y_data.loc[X_Y_data.pid==f'{pid:02}']
                X_Y_data = X_Y_data.loc[X_Y_data.imf == imf].drop(['pid', 'imf'], axis=1)
                Y = np.array([cls_label_trans[cls_key][i] for i in X_Y_data.index])

                X = X_Y_data[Prefrontal_Cortex].values / 4000
                gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
                score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
                ans_df.loc[pid, imf] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Prefronta Cortex_{cls_key}_r{r}.csv", encoding="GBK")
        print("\n", ans_df)

 r 1 class C2 imf gamma pid 20
           delta        theta       alpha         beta        gamma
1    55.6%±5.0%   56.3%±2.7%  61.1%±5.0%   81.7%±9.6%   99.2%±1.4%
2    97.1%±1.3%   97.8%±0.0%  99.3%±1.3%  100.0%±0.0%  100.0%±0.0%
3    62.6%±5.2%  65.6%±13.7%  67.8%±7.7%   87.3%±3.4%   97.7%±2.3%
4    58.7%±2.0%   57.0%±5.7%  67.8%±5.0%   60.3%±4.1%   62.0%±4.2%
5   70.3%±10.7%   87.7%±7.6%  84.8%±3.8%   86.2%±3.3%   95.7%±3.8%
6    76.5%±7.5%   92.1%±8.8%  91.5%±6.4%   70.7%±7.1%   67.8%±3.9%
7    79.7%±6.0%   81.7%±3.0%  88.9%±1.1%   98.7%±1.1%   96.7%±2.3%
8    73.5%±8.2%   68.7%±4.7%  63.3%±4.1%   66.7%±8.2%   64.6%±8.2%
9    63.2%±7.3%   75.0%±3.6%  84.7%±4.3%   67.4%±3.2%   91.0%±3.2%
10   66.2%±4.0%   65.5%±4.6%  89.7%±2.1%   73.8%±4.5%   81.4%±3.5%
11   69.4%±2.4%   65.6%±6.7%  73.1%±2.6%   74.7%±4.4%   82.8%±3.6%
12   49.7%±7.4%   49.6%±5.6%  64.3%±6.7%  75.3%±10.1%  76.7%±13.9%
13   46.7%±8.4%   67.0%±4.3%  57.1%±5.0%  55.8%±15.3%   74.4%±5.0%
14   86.1%±2.0%   92.7%±1.1%  

##### ④.② Visual Cortex

In [28]:
param_grid = [
                {'priors': [None]}
              ]

visual_Cortex = ['Oz','O1','O2','OI1h', 'OI2h']
# 按照半径展开
for r in r_list:
    df = read_ctm(r)

    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        ans_df = pd.DataFrame(columns=imf_list, index=range(1,21))

        if cls_key == 'C2':
            inner_cv = inner_cv_C2
            outer_cv = outer_cv_C2
        else:
            inner_cv = inner_cv_C4
            outer_cv = outer_cv_C4
        # 按照节律展开
        for imf in imf_list:
            # 按照pid 展开
            for pid in range(1, 21):
                sys.stdout.write(f"\r r {r} class {cls_key} imf {imf} pid {pid}")

                X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]

                X_Y_data = X_Y_data.loc[X_Y_data.pid==f'{pid:02}']
                X_Y_data = X_Y_data.loc[X_Y_data.imf == imf].drop(['pid', 'imf'], axis=1)
                Y = np.array([cls_label_trans[cls_key][i] for i in X_Y_data.index])

                X = X_Y_data[visual_Cortex].values/ 4000
                gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
                score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
                ans_df.loc[pid, imf] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Visual Cortex_{cls_key}_r{r}.csv", encoding="GBK")
        print("\n", ans_df)

 r 1 class C2 imf gamma pid 20
           delta       theta        alpha        beta        gamma
1    38.9%±5.0%  61.9%±8.6%   59.5%±4.8%  77.0%±6.0%   66.7%±8.2%
2    52.9%±3.3%  63.8%±5.0%   59.4%±3.3%  44.9%±8.8%  74.6%±10.9%
3    64.9%±7.3%  66.5%±9.9%  73.2%±11.4%  84.4%±5.8%   79.9%±5.7%
4    51.3%±6.9%  44.6%±4.4%   53.7%±3.8%  66.1%±8.8%   67.8%±2.5%
5    68.8%±3.3%  76.8%±3.3%   62.3%±3.3%  73.9%±3.8%   70.3%±3.3%
6    58.6%±9.7%  65.0%±5.1%   72.1%±6.0%  78.5%±9.7%   97.8%±2.2%
7    61.4%±4.1%  58.8%±3.4%   77.1%±4.9%  72.5%±6.8%   68.0%±4.1%
8    57.8%±4.2%  51.7%±6.6%   61.9%±3.1%  57.8%±4.7%  70.7%±10.3%
9    63.9%±6.7%  47.9%±5.5%   65.3%±6.7%  68.8%±4.2%   75.7%±4.8%
10   55.8%±7.0%  55.1%±9.9%  57.2%±10.1%  65.5%±6.6%   85.5%±2.1%
11   68.6%±4.1%  61.2%±6.9%   76.1%±5.0%  76.1%±9.5%   97.8%±0.0%
12  43.7%±13.3%  63.5%±4.4%   56.9%±3.8%  72.3%±6.7%   73.0%±1.6%
13   58.7%±8.8%  61.7%±5.9%   62.4%±3.1%  48.1%±2.8%   85.7%±2.5%
14   51.0%±1.0%  51.6%±5.1%   49.7%±7.4%  55

#### ⑤ 单人5通道多节律分类
##### ⑤.① Prefrontal Cortex

In [29]:
param_grid = [
                {'priors': [None]}
              ]

Prefrontal_Cortex = ['Fpz','Fp1','Fp2','AFp1', 'AFp2']
ans_df = pd.DataFrame()
# 按照半径展开
for r in r_list:
    df = read_ctm(r)
    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():

        if cls_key == 'C2':
            inner_cv = inner_cv_C2
            outer_cv = outer_cv_C2
        else:
            inner_cv = inner_cv_C4
            outer_cv = outer_cv_C4
        # 按照pid 展开
        for pid in range(1, 21):
            sys.stdout.write(f"\r r {r} class {cls_key} pid {pid}")

            X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]
            X_Y_data = X_Y_data[Prefrontal_Cortex+['imf', 'pid']].loc[X_Y_data.pid==f"{pid:02}"].drop(["pid"], axis = 1)
            X_data = pd.DataFrame()
            for imf in imf_list:

                tmp = X_Y_data.loc[X_Y_data.imf == imf]
                tmp = tmp.drop('imf',axis=1)
                tmp.columns = [f"{i}_{imf}" for i in tmp.columns]
                X_data = pd.concat([X_data, tmp], axis=1)
            Y = np.array([cls_label_trans[cls_key][i] for i in X_data.index])
            X = X_data.values/ 4000

            gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
            score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
            # ans_df.loc[pid, f'r={r}_{cls_key}'] = np.mean(score) * 100
            ans_df.loc[pid, f'r={r}_{cls_key}'] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Prefronta Cortex.csv",encoding="GBK")
print("\n", ans_df)

 r 1 class C42 pid 20
          r=1_C2      r=1_C41      r=1_C42
1   100.0%±0.0%  39.3%±15.8%  53.1%±17.1%
2   100.0%±0.0%  28.7%±13.4%  49.1%±20.4%
3    97.7%±2.3%  50.5%±18.4%  40.5%±19.0%
4   66.0%±12.1%  51.2%±14.7%  54.0%±19.2%
5    95.7%±3.8%  50.7%±29.9%  36.2%±15.1%
6    92.1%±5.5%  41.2%±14.6%  50.7%±16.8%
7    99.3%±1.1%  37.1%±20.7%  46.8%±18.8%
8    86.4%±7.7%  53.6%±13.7%  42.9%±19.0%
9    92.4%±1.2%  48.6%±18.1%  53.4%±14.9%
10   90.3%±7.3%  38.9%±16.9%  51.4%±13.8%
11   82.1%±4.0%  42.9%±16.1%  62.9%±17.9%
12   78.9%±9.7%  51.4%±24.9%  48.1%±15.3%
13   72.9%±0.3%  45.5%±23.3%  42.6%±18.3%
14  100.0%±0.0%  50.9%±16.2%  53.9%±14.5%
15   71.5%±6.7%  43.8%±18.1%  48.6%±22.5%
16   81.8%±9.1%  54.3%±13.9%   59.3%±9.6%
17  100.0%±0.0%  60.0%±22.5%  54.8%±18.7%
18  83.8%±14.9%  39.0%±19.4%  51.4%±13.6%
19   86.9%±5.5%  56.0%±19.5%  38.8%±20.7%
20   94.6%±5.9%  52.1%±17.9%  38.8%±17.9%


##### ⑤.② Visual Cortex

In [30]:
param_grid = [
                {'priors': [None]}
              ]

visual_Cortex = ['Oz','O1','O2','OI1h', 'OI2h']
ans_df = pd.DataFrame()
# 按照半径展开
for r in r_list:
    df = read_ctm(r)
    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():

        if cls_key == 'C2':
            inner_cv = inner_cv_C2
            outer_cv = outer_cv_C2
        else:
            inner_cv = inner_cv_C4
            outer_cv = outer_cv_C4
        # 按照pid 展开
        for pid in range(1, 21):
            sys.stdout.write(f"\r r {r} class {cls_key} pid {pid}")

            X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]
            X_Y_data = X_Y_data[visual_Cortex+['imf', 'pid']].loc[X_Y_data.pid==f"{pid:02}"].drop(["pid"], axis = 1)
            X_data = pd.DataFrame()
            for imf in imf_list:
                tmp = X_Y_data.loc[X_Y_data.imf == imf]
                tmp = tmp.drop('imf',axis=1)
                tmp.columns = [f"{i}_{imf}" for i in tmp.columns]
                X_data = pd.concat([X_data, tmp], axis=1)
            Y = np.array([cls_label_trans[cls_key][i] for i in X_data.index if i in cls_label_trans[cls_key].keys()])
            X = X_data.values/ 4000

            gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
            score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
            # ans_df.loc[pid, f'r={r}_{cls_key}'] = np.mean(score) * 100
            ans_df.loc[pid, f'r={r}_{cls_key}'] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"
        ans_df.to_csv(f"{result_path}\\result4_20people_visual Cortex.csv", encoding="GBK")
print("\n", ans_df)

 r 1 class C42 pid 20
          r=1_C2      r=1_C41      r=1_C42
1    77.0%±6.0%  34.5%±14.7%  41.4%±13.8%
2    79.0%±8.8%  66.3%±15.4%  55.7%±13.3%
3    91.1%±5.9%  39.1%±19.7%  59.0%±18.1%
4    67.8%±6.5%  53.6%±26.9%  48.3%±21.0%
5    82.6%±7.5%  63.3%±21.6%  61.3%±13.5%
6    95.7%±2.1%  67.1%±13.7%  60.9%±12.0%
7    80.4%±5.2%  42.9%±14.6%  38.4%±12.0%
8    70.1%±5.9%  54.6%±10.4%  52.9%±21.3%
9    74.3%±2.4%  58.6%±21.8%  48.6%±12.5%
10   86.9%±5.9%  52.3%±15.3%  52.9%±19.1%
11   94.7%±3.5%  60.5%±17.9%  59.0%±22.2%
12   71.6%±5.6%  52.0%±29.1%  43.3%±17.6%
13   84.9%±2.7%  59.8%±17.0%  54.0%±18.2%
14  71.6%±15.6%  50.4%±19.0%  45.2%±18.3%
15   68.8%±5.5%  47.9%±20.1%  47.1%±13.6%
16   92.6%±3.0%  38.0%±20.5%  34.3%±24.5%
17   70.2%±6.8%  43.3%±21.1%  51.4%±20.9%
18   94.9%±0.1%  44.7%±21.2%  59.3%±20.2%
19  74.6%±10.7%  48.8%±17.2%  49.3%±21.4%
20   80.3%±7.7%  46.2%±18.7%  59.8%±18.8%
